In [11]:
import subprocess
# Check if packages are installed
def check_packages():
    try:
        with open('requirements.txt', 'r') as file:
            packages = file.read().splitlines()
            print(f"Packages to check: {packages}")  # Debug print
            for package in packages:
                try:
                    __import__(package)
                    print(f"{package} is already installed.")
                except ImportError:
                    print(f"{package} is not installed. Installing...")
                    subprocess.check_call(['pip', 'install', package])
                    print(f"{package} has been installed.")
    except Exception as e:
        print(f"An error occurred: {e}")  # Print any other errors

# Call the function to check and install packages
check_packages()

Packages to check: ['requests', 'aiohttp', 'pandas', 'tweepy', 'praw', 'ccxt', 'transformers', 'vaderSentiment', 'textblob', 'flask', 'plotly', 'beautifulsoup4']
requests is already installed.
aiohttp is already installed.
pandas is already installed.
tweepy is already installed.
praw is already installed.
ccxt is already installed.


c:\Users\UrsHu\Pillars\Learn\Academic\Master\Semestre 2\data analysis\crypto-sentiment-tracker\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


transformers is already installed.
vaderSentiment is already installed.
textblob is already installed.
flask is already installed.
plotly is already installed.
beautifulsoup4 is not installed. Installing...
beautifulsoup4 has been installed.


In [13]:
import requests
from bs4 import BeautifulSoup

# Example URL
url = 'http://twitter.com/'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find the login button
login_button = soup.find('a', {'href': '/login', 'role': 'link', 'class': 'css-175oi2r r-sdzlij r-1phboty r-rs99b7 r-lrvibr r-17w48nw r-a9p05 r-eu3ka r-5oul0u r-1ipicw7 r-2yi16 r-1qi8awa r-ymttw5 r-o7ynqc r-6416eg r-1ny4l3l r-1loqt21'})

# Click on the login button
login_button.click()


AttributeError: 'NoneType' object has no attribute 'click'